In [1]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim import utils
# random
import random

# numpy
import re
import numpy
import gensim, logging
import os
import jieba
import json

# classifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import  LinearSVC
from config import *

In [2]:
def filter(line):
    #剔除日期
    data_regex = re.compile(u"""        #utf-8编码
        年 |
        月 |
        日 |
        (周一) |
        (周二) | 
        (周三) | 
        (周四) | 
        (周五) | 
        (周六)
    """, re.VERBOSE)
    #剔除所有数字
    decimal_regex = re.compile(r"[^a-zA-Z]\d+")

    line = data_regex.sub(r"", line)
    line = decimal_regex.sub(r"", line)

    return line

def trans(sentence,cutwordslist = None):
    texts_cut = [word for word in jieba.lcut(filter(sentence)) if len(word) > 1]
    outstr = ' '.join(texts_cut)
    return outstr

In [3]:
def saveTxt():
    se = set()
    f = open("../input/input.txt", "r", encoding="utf8")
    for line in f:
        x = json.loads(line)
        se.add(x["A"])
        se.add(x["B"])
        se.add(x["C"])
        
    new_file = open("../input/new_input.txt", 'a',encoding="utf8")
    cutwordslist = []
    data = list(se)
    for a in range(0, len(data)):
        data[a] = trans(data[a],cutwordslist)
        
        
        #打开fie_name路径下的my_infor.txt文件,采用追加模式
        #若文件不存在,创建，若存在，追加
        new_file.write(data[a] + '\n')
    new_file.close()

In [4]:
saveTxt()

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/8j/p8231m8944s1l43bpl8h5t780000gn/T/jieba.cache
Loading model cost 0.671 seconds.
Prefix dict has been built succesfully.


In [5]:
class TaggedLineSentence(object):
    def __init__(self, sources):
        self.sources = sources
        flipped = {}
        self.tagToText = dict()

        # make sure that keys are unique
        for key, value in sources.items():
            if value not in flipped:
                flipped[value] = [key]
            else:
                raise Exception('Non-unique prefix encountered')

    def __iter__(self):
        for source, prefix in self.sources.items():
            with utils.smart_open(source) as fin:
                for item_no, line in enumerate(fin):
                    yield TaggedDocument(utils.to_unicode(line).split(), [prefix + '_%s' % item_no])

    def to_array(self):
        self.sentences = []
        for source, prefix in self.sources.items():
            with utils.smart_open(source) as fin:
                for item_no, line in enumerate(fin):
                    text = utils.to_unicode(line).split()
                    tag = prefix + '_%s' % item_no
                    self.sentences.append(TaggedDocument(text, [tag]))
                    self.tagToText[tag] = text
        return(self.sentences)

    def sentences_perm(self):
        shuffled = list(self.sentences)
        random.shuffle(shuffled)
        return(shuffled)

In [6]:
sources = {'../input/new_input.txt':'TRAIN'}

sentences = TaggedLineSentence(sources)
model = Doc2Vec(min_count=1, window=5, vector_size=300, sample=1e-4, negative=5, workers=7)
model.build_vocab(sentences.to_array())
model.train(sentences.sentences_perm(), epochs=500, total_examples=model.corpus_count)

In [7]:
model.wv.most_similar('贷款')  

[('同期', 0.3399195075035095),
 ('按分', 0.27054232358932495),
 ('房地产', 0.27041709423065186),
 ('一切', 0.2581440210342407),
 ('建筑', 0.25032883882522583),
 ('以上', 0.24582038819789886),
 ('连带责任', 0.24567046761512756),
 ('嘉合', 0.23965546488761902),
 ('特此', 0.22511425614356995),
 ('暨阳', 0.2226559817790985)]

In [8]:
prefix_train_pos = 'TRAIN_' + str(0)
model[prefix_train_pos]

array([ 0.24723788, -0.2669648 ,  0.52133167,  0.04069306,  0.05110539,
        1.3481535 ,  0.36120468, -0.5461391 ,  0.07782156, -0.30396992,
       -0.25895113,  0.39769256,  0.48989594,  0.01740984,  0.88850015,
        0.3721652 , -0.40815273,  0.10338775,  0.6010039 ,  0.13516295,
        0.76315945,  0.0016245 ,  0.51331794,  0.04475264,  0.2412806 ,
        0.6342639 ,  0.6985495 ,  0.9689609 ,  0.30470052, -0.33417118,
        0.06756281,  0.09594025,  0.50065637, -0.50017506,  0.83391696,
       -0.54454964, -1.0243362 ,  0.18816279, -0.11972433,  0.20329353,
       -0.38969728,  0.84276843,  0.53447324,  1.0577382 , -0.4552966 ,
       -0.5803535 , -0.03164423,  0.39466655,  0.40108457,  0.39115265,
        0.2858232 ,  1.0854676 , -0.9312225 ,  0.65878797, -0.37902713,
        0.68307203,  0.4871506 , -0.14926112,  0.3415158 ,  0.48316178,
       -0.14005838, -0.76269644, -0.39544892, -0.59608567,  0.337287  ,
        0.40987593, -0.30730993,  0.49860027,  0.32741287, -0.12

In [9]:
print(' '.join(sentences.tagToText[prefix_train_pos]))

原告 出生 汉族 农民 蒲城县 山镇 被告 出生 汉族 农民 蒲城县 山镇 原告 屈向 本院 提出 诉讼请求 请求 判令 被告 立即 清偿 原告 借款 本元及 利息 按利 计算 清偿 案件 受理费 被告 承担 事实 理由 被告 同村 村民 被告 安以 生意 资金周转 困难 原告 借元 双方 口头 约定 借款 期限 借款 到期 被告 并未 履行 还款 义务 被告 安未作 答辩 原告 支持 主张 本院 提供 被告 署名 借条 一张 屈晓辉 证人 证言 用以 证明 被告 原元 事实 被告 安未 提供 证据 被告 安经 传唤 到庭 提供 答辩 质证 意见 视为 自己 质证 权利 放弃 原告 提供 证据 真实 合法 两份 证据 之间 能够 相互 印证 可以 作为 定案 依据 根据 认定 证据 以及 当事人 庭审 陈述 能够 查明 以下 案件 事实 被告 曾系 同事 关系 被告 资金周转 困难 原告 借款 原告 当时 旬阳 尧柏 水泥厂 上班 支付宝 转账 形式 案外人 屈晓辉 转元 委托 屈晓辉 交于 被告 被告 原告 出具 借条 屈晓辉 交于 被告 被告 原告 出具 借条 一张 交由 屈晓辉 内容 借条 今借 屈壹 万元 整元整 身份证号 借款 时未 约定 利息 双方 口头 约定 还款 期限


In [10]:
text = ' '.join(sentences.tagToText[prefix_train_pos])
tokens = text.split()
print(tokens)

['原告', '出生', '汉族', '农民', '蒲城县', '山镇', '被告', '出生', '汉族', '农民', '蒲城县', '山镇', '原告', '屈向', '本院', '提出', '诉讼请求', '请求', '判令', '被告', '立即', '清偿', '原告', '借款', '本元及', '利息', '按利', '计算', '清偿', '案件', '受理费', '被告', '承担', '事实', '理由', '被告', '同村', '村民', '被告', '安以', '生意', '资金周转', '困难', '原告', '借元', '双方', '口头', '约定', '借款', '期限', '借款', '到期', '被告', '并未', '履行', '还款', '义务', '被告', '安未作', '答辩', '原告', '支持', '主张', '本院', '提供', '被告', '署名', '借条', '一张', '屈晓辉', '证人', '证言', '用以', '证明', '被告', '原元', '事实', '被告', '安未', '提供', '证据', '被告', '安经', '传唤', '到庭', '提供', '答辩', '质证', '意见', '视为', '自己', '质证', '权利', '放弃', '原告', '提供', '证据', '真实', '合法', '两份', '证据', '之间', '能够', '相互', '印证', '可以', '作为', '定案', '依据', '根据', '认定', '证据', '以及', '当事人', '庭审', '陈述', '能够', '查明', '以下', '案件', '事实', '被告', '曾系', '同事', '关系', '被告', '资金周转', '困难', '原告', '借款', '原告', '当时', '旬阳', '尧柏', '水泥厂', '上班', '支付宝', '转账', '形式', '案外人', '屈晓辉', '转元', '委托', '屈晓辉', '交于', '被告', '被告', '原告', '出具', '借条', '屈晓辉', '交于', '被告', '被告', '原告', '出具', '借条', '一张', '交由', '屈晓辉', '内容', '借条', '今借', '

In [11]:
new_vector = model.infer_vector(tokens)

In [12]:
model.save(os.path.join(MODEL_PATH,DOC2VEC))

In [13]:
model = Doc2Vec.load(os.path.join(MODEL_PATH,DOC2VEC))  # you can continue training with the loaded model!